In [1]:
!pip install requests
!pip install gradio rapidfuzz wikipedia openai
import os
import gradio as gr
import random
import openai
import wikipedia
import requests
from rapidfuzz import fuzz, process

# Set OpenAI API key
openai.api_key = os.getenv(" ")

# Cheer up messages
cheer_up_messages = [
    "You're amazing, just the way you are! 💖",
    "Hey, it's okay to have a rough day. You're doing great! 🌈",
    "Believe in yourself. You've got this! 💪",
    "Sending virtual hugs your way! 🤗",
    "You're stronger than you think. Keep going! 🌟",
    "Smile! You're someone’s reason to be happy. 😊",
    "Storms don’t last forever. Brighter days are ahead! ☄️",
    "You've already survived 100% of your worst days. Go you! 🎉"
]

# Average package data
average_package_data = {
    "cse": "Computer Science and Engineering: ₹5.80 LPA",
    "computer science": "Computer Science and Engineering: ₹5.80 LPA",
    "ece": "Electronics and Communication Engineering: ₹5.20 LPA",
    "electronics": "Electronics and Communication Engineering: ₹5.20 LPA",
    "mechanical": "Mechanical Engineering: ₹5.00 LPA",
    "civil": "Civil Engineering: ₹4.80 LPA",
    "electrical": "Electrical Engineering: ₹4.60 LPA",
    "biotech": "Biotechnology: ₹4.40 LPA",
    "biotechnology": "Biotechnology: ₹4.40 LPA",
    "instrumentation": "Instrumentation and Control Engineering: ₹4.20 LPA",
    "aeronautical": "Aeronautical Engineering: ₹4.00 LPA",
    "mining": "Mining Engineering: ₹3.80 LPA",
    "architecture": "Architecture: ₹3.60 LPA",
    "mba": "MBA/PGDM: ₹6.00 LPA",
    "pgdm": "MBA/PGDM: ₹6.00 LPA",
    "bba": "BBA: ₹3.60 LPA",
    "computer applications": "Computer Applications: ₹4.00 LPA",
    "law": "Law: ₹3.50 LPA",
    "media": "Media & Communication: ₹3.80 LPA",
    "media & communication": "Media & Communication: ₹3.80 LPA"
}

# Intent patterns
intent_patterns = {
    "admission": ["admission", "apply amity", "get into amity", "amity application", "how to join amity"],
    "syllabus": ["syllabus", "subjects", "per sem", "semester subjects", "course structure"],
    "notes": ["notes", "previous year paper", "pyq", "past question paper", "aminotes", "amity test"],
    "about": ["about amity", "amity university", "tell me about amity", "amity info"],
    "why_join": ["why join amity", "why amity", "better amity", "reasons to choose amity"],
    "student_portal": ["student portal", "amizone", "amizone login", "amity portal"],
    "cheer_up": ["cheer", "motivate", "sad", "feeling low", "mental health", "unhappy", "depressed"],
    "average_package_all": ["amity average package", "average package", "amity package", "packages", "placement package"],
    "joke": ["tell me a joke", "make me laugh", "funny", "joke please"]
}


# Match intent
def match_intent(user_input, threshold=60):
    best_intent = None
    best_score = threshold
    for intent, patterns in intent_patterns.items():
        for pat in patterns:
            score = fuzz.token_set_ratio(user_input, pat)
            if score > best_score:
                best_score = score
                best_intent = intent
    return best_intent

# Match branch
def match_branch(user_input, threshold=60):
    match, score, _ = process.extractOne(
        user_input, average_package_data.keys(), scorer=fuzz.token_set_ratio
    )
    if score >= threshold:
        return match
    return None

# Joke API function
def get_joke():
    try:
        response = requests.get("https://v2.jokeapi.dev/joke/Any?format=txt")
        if response.status_code == 200:
            return f"😂 Here's a joke for you:\n\n{response.text}"
        else:
            return "❌ Couldn't fetch a joke right now."
    except Exception:
        return "❗ Error fetching joke."

# DuckDuckGo search fallback
def duckduckgo_search(query):
    try:
        url = f"https://api.duckduckgo.com/?q={query}&format=json"
        res = requests.get(url).json()
        if res.get("AbstractText"):
            return f"🔎 DuckDuckGo says:\n\n{res['AbstractText']}"
        elif res.get("RelatedTopics"):
            for topic in res["RelatedTopics"]:
                if topic.get("Text"):
                    return f"🔎 DuckDuckGo says:\n\n{topic['Text']}"
        return None
    except:
        return None

# General question answering
def general_qa(user_input):
    # Try DuckDuckGo
    ddg_response = duckduckgo_search(user_input)
    if ddg_response:
        return ddg_response

    # Try Wikipedia
    try:
        summary = wikipedia.summary(user_input, sentences=2)
        return f"🔎 Wikipedia says:\n\n{summary}"
    except:
        pass

    # Fallback joke
    if "joke" in user_input.lower():
        return get_joke()

    # GPT fallback
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": user_input}],
            temperature=0.7,
            max_tokens=200
        )
        return response.choices[0].message.content.strip()
    except:
        return "🤔 I couldn't find an answer to that. Try asking something else?"

# Main response function
def amibot_response(user_input):
    ui = user_input.lower()

    # If user mentions joke directly
    if "joke" in ui:
        return get_joke()

    intent = match_intent(ui)

    if intent == "admission":
        return "🎓 To apply at Amity University, visit: [Admission Portal](https://portal.amity.edu/)"

    elif intent == "syllabus":
        return "📘 Check the syllabus and subjects here: [Amity Syllabus](https://www.amity.edu)"

    elif intent == "notes":
        return (
            "📝 Notes & Resources:\n"
            "- [Aminotes](https://www.aminotes.com)\n"
            "- [Google Drive PYQs](https://amityedu96491-my.sharepoint.com/:f:/g/personal/devansh_agarwal4_s_amity_edu/EtZHENaqcLtJsRR1d2dNMaQBrzDyiZgfyK1ZKzQIBclzYg)"
        )

    elif intent == "about":
        return (
            "🏫 **Amity University** is a top private institution in India.\n\n"
            "- NIRF 2023 Rankings:\n"
            "   - Overall: 49th\n   - Engineering: 30th\n   - Pharmacy: 21st\n"
            "- Collaborates with global unis & top recruiters.\n"
            "- Visit: [amity.edu](https://www.amity.edu)"
        )

    elif intent == "why_join":
        return (
              """
Here’s a solid list of reasons **why you might consider joining Amity University**—especially if you're looking for a well-rounded educational experience:

---

### 🎓 **1. Strong Academic Programs**
Amity offers a wide range of **UG, PG, and PhD** programs across disciplines—engineering, business, law, psychology, media, biotech, design, etc. Their curriculum is regularly updated to match industry needs.

---

### 🌐 **2. Global Exposure**
- Tie-ups with **international universities** like Stanford, Purdue, and NYU.
- Opportunities for **student exchange programs**, summer schools, and global internships.
- Many faculties are **PhD holders or industry professionals** with international exposure.

---

### 🏫 **3. Infrastructure & Campus Life**
- **High-tech campuses** with smart classrooms, innovation labs, sports complexes, and libraries.
- Active student clubs, hackathons, MUNs, and cultural festivals like **Sangathan** & **Amiphoria**.
- On-campus hostels with decent facilities and a focus on student safety.

---

### 💼 **4. Placement & Career Support**
- **Centralized placement cell** with tie-ups to top recruiters like TCS, Amazon, Accenture, Flipkart, and more.
- Regular workshops, soft skill training, and internships from **first year onward**.
- Many students from tech, law, and management branches get placed in good companies or opt for higher studies abroad.

---

### 🧠 **5. Innovation & Entrepreneurship Support**
- **Amity Innovation Incubator** supports startup ideas.
- Dedicated research centers in AI, cybersecurity, robotics, finance, and more.
- Encourages students to file **patents and publish research**.

---

### ❤️ **6. Personality Development Focus**
- Courses on **communication, ethics, body language**, and leadership.
- Events and platforms to build **confidence, creativity, and public speaking skills**.
- A vibrant, extrovert-friendly atmosphere (which you’d vibe with perfectly 😉).

---

### ✨ Bonus: Unique Offerings
- Amity runs an **Online Degree Program** too (flexibility!)
- Top-notch **PhD programs** and UGC-recognized degrees.
- Focus on **holistic development**: academics + mental health + extracurriculars.

---

If you’re looking for an institution that mixes **academic rigor**, **global standards**, **industry exposure**, and a **dynamic student life**, Amity can be a strong choice.
"""
        )

    elif intent == "student_portal":
        return "🔗 Access the Amizone Portal here: [Amizone](https://s.amizone.net)"

    elif intent == "cheer_up":
        return random.choice(cheer_up_messages)

    branch_key = match_branch(ui)
    if branch_key and ("package" in ui or "average package" in ui or "placement" in ui):
        return f"💼 {average_package_data[branch_key]}"

    if intent == "average_package_all":
        lines = ["💼 **Amity Average Package (Noida Campus):**"]
        for val in average_package_data.values():
            lines.append(f"- {val}")
        return "\n".join(lines)

    return general_qa(user_input)

# Gradio UI
chat_ui = gr.Interface(
    fn=amibot_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask me about Amity, placements, syllabus, jokes, or anything else!"),
    outputs="markdown",
    title="🤖 Amibot – Your Amity + Fun Assistant",
    description="Ask about Amity University, get notes/placements, or let me make you laugh with a joke!"
)

if __name__ == "__main__":
    chat_ui.launch()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.0 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=14ac8033ea8b10cd7ca068ccc9573995df247400fd0c1d53d687511376896f47
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://932b34318f23a44f85.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
